In [ ]:
from langchain.memory import ConversationBufferMemory

#predict시 유용한 memory. NOT economic. Just saves the text accumulatively.
memory = ConversationBufferMemory()#return_messages: False가 default. Conversation은 True로 변경해야 한다.

memory.save_context({"input":"Hi"},{"AI": "How are you?"})

memory.load_memory_variables({})

#Conver

{'history': 'Human: Hi\nAI: How are you?'}

In [ ]:
# K만큼의 대화만 저장하는 memory
from langchain.memory import ConversationBufferWindowMemory

memory= ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("1","1")


memory.load_memory_variables({})

{'history': [HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={})]}

In [ ]:
#이전 대화를 요약해서 저장하는 memory
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi, I'm Nicholas. I live in South Korea","Wow that is so cool!")

add_message("South Korea is so pretty","I wish I could go")

get_history()

C:\Users\dlaru\AppData\Local\Temp\ipykernel_7028\2897872419.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.1)
C:\Users\dlaru\AppData\Local\Temp\ipykernel_7028\2897872419.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [ ]:
# 일정 token 이상의 대화는 요약해서 저장하는 memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
import tiktoken

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory (
    llm=llm,
    max_token_limit=10,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi, I'm Nicholas. I live in South Korea","Wow that is so cool!")

add_message("South Korea is so pretty","I wish I could go")

get_history()

{'history': [SystemMessage(content='Nicholas introduces himself as living in South Korea. The AI responds with enthusiasm at the information, expressing a desire to visit the country because it is so pretty.', additional_kwargs={}, response_metadata={})]}

In [7]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI
import networkx


llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory (
    llm=llm,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hi, I'm Nicholas. I live in South Korea","Wow that is so cool!")

memory.load_memory_variables({"input":"Who is Nicholas?"})




{'history': [SystemMessage(content='On Nicholas: Nicholas lives in South Korea.', additional_kwargs={}, response_metadata={})]}

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

# Designate variables
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o"
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages= True,
    memory_key="chat_history"
)

# Load and split document
loader = TextLoader(r"./files/chapter_3.txt")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

# Set up embeddings and vectorstore
embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./cache/")
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir    
)
vectorstore = Chroma.from_documents(docs, cache_embeddings)
retriever = vectorstore.as_retriever()

# List of docs generation
map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim
    ------------------
    {portion}
    """),
    ("human", "{question}"),
])
map_doc_chain = map_doc_prompt | llm

# Merge list of docs
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(map_doc_chain.invoke({
        "portion": doc.page_content,
        "question": question
    }).content for doc in documents)

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

# Final document | prompt | llm
final_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Give the following extracted parts of a long document and a question, create a final answer.
    If you don't know the answer, just say that you don't know. Don't try to make up the answer.
    ---------
    {context}
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]



chain =  ???

def invoke_chain(question):
    result=chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

invoke_chain("Is Aaronson guilty?")


ImportError: cannot import name 'RunnableMap' from 'langchain.chains' (c:\Users\dlaru\OneDrive\바탕 화면\fullstack_gpt\env\Lib\site-packages\langchain\chains\__init__.py)